In [1]:
####### Use Selenium to open webpages that use java script rendering ###############
from selenium import webdriver
####### Use Pandas to manage and arrange data #################
import pandas as pd
####### Use BeautifulSoup to read and navigate the DOM ################
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import urllib.error
######### Use Regex to clean up some of the elements we scrape #################
import re
########## Use Numpy for data manipulation ###############
import numpy as np
########## Use the nltk library to tokenize and analyse text results ##########
import nltk
from nltk.tokenize import sent_tokenize
############# Use Networks for graphing of text analysis #############
import networkx as nx
############# use Matplotlib and WordCloud to create and display a wordcloud ################
import matplotlib as plt
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
############## Download these files if they are not upto date #############
# nltk.download('stopwords')
# nltk.download('punkt')

In [1]:
############# First we ask user for input of what topics should be searched/how many pages of Google reuslts they would like/ and how recent the results should be limited to ######################
results = []
pages = []
allsellers = []
links = []
terms = input('Enter what you would like to search : ')
terms = terms.split(', ')
length = input('How many pages would you like to search : ')
length = int(length)
recency = input('How recently do you want results from? Select h/d/w/m/y :  ')
recency = str(recency)
i = 0
############# for each search term entered run the for loop to gather the search results #########
############# 'i' represents the number of pages requested then a decreasing while loop will gather the needed reults ###############
for t in terms:
    while i < length:
        i += 1
        pnum = str(1-i)
        driver = webdriver.Chrome()
        base = 'https://www.google.com/search?q='
        time = ('&tbs=qdr:'+recency)
        depth = '&start='+pnum+'0'
        url = base+t+time+depth
        driver.implicitly_wait(30)
        driver.get(url)
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        results=soup.find_all('div',{'class':'r'})
        
        for link in results:
            links.extend(link.find_all('a'))
        links=[re.sub(r'.*href\=\"',r'',str(a))for a in links]
        links=[re.sub(r'".*',r'',str(a))for a in links]
        while '#' in links:
            links.remove('#')
        driver.quit()
############ once we have gathered all of the links from the pages into a list we can clean them and remove unwanted domians ###########        
links = [re.sub(r'.*https',r'https',str(a))for a in links]
links = [re.sub(r'\/search\?q\=related\:',r'https\:\/\/',str(a))for a in links]
links = [x for x in links if 'wikipedia' not in x]
links = [x for x in links if 'encyclopedia' not in x]
print(len(links))
links = [re.sub(r'\+\&amp.*',r'',str(a))for a in links]
links=set(links)
print(len(links))

In [2]:
################ this function will look at the parent element tags and filter out ones that are irrelivant #############
def visable(element):
    if element.parent.name in ['style', 'script', ['document'], 'head', 'title', 'meta']:
        return False
    elif re.match(r'<!--.*-->',str(element.encode('utf-8'))):
        return False
    return True

In [3]:
print(links)

In [4]:
############### Now that the links have been cleaned we can open each page and scrape the 'visible' elements for the text #################
corpus = []
art = links
for url in art:
    ################## we first try to access the page with BeautifulSoup because it is faster #############
    ################## if it errors out we will try to open it with Selenium to render javascript ############
    ################## if both methods fail we will skip the page ###############
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')
        data = soup.findAll(test=True)
        result = filter(visable, data)
        result = [re.sub(r'\n',r'',str(a))for a in result]
        result = [re.sub(r'.*src\=.*',r'',str(a))for a in result]
        result = [re.sub(r'\t',r'',str(a))for a in result]
        result = [re.sub(r'\<.*\>',r'',str(a))for a in result]
        result = [re.sub(r'\{.*\}',r'',str(a))for a in result]
        while '' in result:
            result.remove('')
        corpus.appen(result)
    except:
        driver=webdriver.Chrome()
        driver.implicitly_wait(30)
        try:
            driver.get(url)
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            driver.quit()
            data = soup.findAll(text = True)

            result = filter(visable, data)
            result = [re.sub(r'\n',r'',str(a))for a in result]
            result = [re.sub(r'.*src\=.*',r'',str(a))for a in result]
            result = [re.sub(r'\t',r'',str(a))for a in result]
            result = [re.sub(r'\<.*\>',r'',str(a))for a in result]
            result = [re.sub(r'\{.*\}',r'',str(a))for a in result]
            while '' in result:
                result.remove('')
            corpus.append(result)
        except:
            result = 'Bad URL at : ' + url
            corpus.append(result)
print(len(corpus))

In [77]:
text=corpus

In [78]:
############# in an effort to remove visible text that is unrealted to the page content, like social media links and ads, we filter out strings less than 100 charcters. ##############
############# NOTE: this may also remove <li> elements or other relevant but very short elements on the page ###############
final = []
for i in text:
    temp = []
    for x in i:
#         print(len(x))
        s = len(x)
        if s > 100:
#             y = [re.sub(r'.*',r'',str(a))for a in x]
#             while '' in y:
#                 y.remove('')
            temp.append(x)
        else:
            x=1
    final.append(temp)

In [5]:
print(final)

In [79]:
############## Create a single string per page ##############
df = [''.join(x)for x in final]

In [80]:
############# Tokenize the results ########################
sentences = []
for s in df:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [6]:
print(len(sentences))

In [60]:
########## Extract word vectors from GloVe file ##################3
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [82]:
##################### remove punctuations, numbers and special characters ####################
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

############ make alphabets lowercase ##################
clean_sentences = [s.lower() for s in clean_sentences]

In [62]:
############## import stopwords  list (or create your own)#####################
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [63]:
##################### this function will remove stopwords ####################
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [83]:
############## remove stopwords from the sentences ######################
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [84]:
########### create a vector for the sentences using the GloVe vector library and pulled text ##############
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [85]:
############## create a similarity matrix ################
sim_mat = np.zeros([len(sentences), len(sentences)])

In [86]:
from sklearn.metrics.pairwise import cosine_similarity

In [87]:
########## perform cosine-similarity for vectors #############
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]


In [88]:
########### rank pages based on cosine similarity ##########
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [89]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [7]:
############ Extract top 10 sentences as the summary #################
for i in range(10):
  print(ranked_sentences[i][1])

In [91]:
import matplotlib.pyplot as plt

In [8]:
########### create word cloud for ranked sentences #############
wctext = ranked_sentences
wctext = str(wctext)
wordcloud = WordCloud().generate(wctext)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show